In [1]:
import pandas as pd

In [2]:

def get_only_uniprot_accession(df, colname):
    
    colname_out = colname + '_uniprot_accession'
    
    df[colname_out] = df[colname].str.split('-', expand=True)[1]
    
    return df


def calc_coverage(df, which_one ,start, end, lenght):
    
    col_name = 'COV_' + which_one
    
    df[col_name] = (df.loc[: , end] - df.loc[: , start]) / df.loc[: , lenght]
    
    df[col_name] = df[col_name].round(2)
    
    return df

## Importing df 

In [3]:
df = pd.read_csv('../../../../results/foldseek_search_TSV/cluster_representer_vs_SwissProt.tsv', 
            sep='\t', 
            names = ['query','target','alnlen','fident', 'evalue', 'qstart','qend', 'qlen','tstart','tend','tlen','aligmentinfo'])

## modifing columns

In [4]:
df = (df
 .pipe(get_only_uniprot_accession, 'query')
 .pipe(get_only_uniprot_accession, 'target')
 .pipe(calc_coverage, 'query', 'qstart', 'qend','qlen')
 .pipe(calc_coverage, 'target', 'tstart', 'tend','tlen'))

print(df.shape)

df = df.drop_duplicates(['query_uniprot_accession', 'target_uniprot_accession'])

print(df.shape)

(1504223, 16)
(1504223, 16)


## Amount of cluster with rbh result 

In [5]:
df_ogup = pd.read_csv('../../../report/ortholog_groups_x_sequence_clustering_x_UNIPROT.tsv', sep='\t')

df_ogup = df_ogup[~df_ogup.uniprot.isna()]

print('Total amount of cluster to annotate:')
print(df_ogup.Ortholog_Group.nunique())

Total amount of cluster to annotate:
14998


In [6]:
#merging df 
df_merged = df_ogup.merge(df, left_on='uniprot', right_on='query_uniprot_accession', how='right')

In [7]:
df_merged[
    (df_merged['COV_query'] > .7) &
    (df_merged['COV_target'] > .7) &
    (df_merged['evalue'] < 1e-5)].sort_values('evalue').Ortholog_Group.nunique()

6047

In [10]:
df.query_uniprot_accession.nunique()

8811